### Instalacja bibliotek

In [ ]:
pip install tensorflow

### Importowanie potrzebnych bibliotek

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


### Definicja modelu

In [ ]:
model = Sequential([
    # Pierwsza warstwa konwolucyjna
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    # Druga warstwa konwolucyjna
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # Trzecia warstwa konwolucyjna
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # Spłaszczenie wyników do jednowymiarowego wektora
    Flatten(),
    # Warstwa gęsta do klasyfikacji
    Dense(512, activation='relu'),
    # Warstwa wyjściowa
    Dense(1, activation='sigmoid')
])

### Kompilacja modelu

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Podsumowanie modelu

In [ ]:
model.summary()

### Importowanie bibliotek do testów

In [ ]:
# Importowanie dodatkowych niezbędnych bibliotek
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Ścieżki do danych treningowych i walidacyjnych


In [ ]:
train_data_path = 'Train-photos'
test_data_path = 'Validation-photos'

### Przygotowanie danych

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

### Przepływ danych treningowych i walidacyjnych

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_data_path,
        target_size=(64, 64), # Jak w input_shape modelu
        batch_size=32,
        class_mode='binary') # Dla klasyfikacji binarnej

validation_generator = validation_datagen.flow_from_directory(
        validation_data_path,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

### Trenowanie modelu

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,  # Batchy na epokę
      epochs=15,  # Liczba epok
      validation_data=validation_generator,
      validation_steps=50,  # Liczba batchy walidacyjnych na epokę
      verbose=2)  # Pokazuje postęp dla każdej epoki

### Ocena modelu na danych testowych

In [ ]:
test_data_path = 'ścieżka_do_danych_testowych'
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_data_path,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

test_loss, test_accuracy = model.evaluate(test_generator)
print('Dokładność na danych testowych:', test_accuracy)

### Zapisanie modelu

In [ ]:
model.save('model_face_detection.h5')

### Użycie modelu do predykcji

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

### Załadowanie obrazu do sklasyfikowania

In [ ]:
test_image = image.load_img('Test-photos/photo-test.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

### Predykcja obrazu

In [ ]:
prediction = model.predict(test_image)
print('Predykcja:', prediction)

### Importowanie dodatkowych bibliotek

In [ ]:

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

### Callbacks

In [ ]:
checkpoint = ModelCheckpoint('model_face_detection_best.h5', 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')
early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5, 
                               verbose=1, 
                               mode='min')

callbacks_list = [checkpoint, early_stopping]

### Trenowanie modelu z callbacks


In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2,
      callbacks=callbacks_list)

### Wykresy dla dokładności i straty

In [ ]:
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

plot_training_history(history)

### Ładowanie najlepszego modelu i testowanie na nowym obrazie


In [ ]:
from tensorflow.keras.models import load_model

### Ładowanie zapisanego modelu


In [ ]:
saved_model = load_model('model_face_detection_best.h5')

### Przykład predykcji na nowym obrazie

In [ ]:
def make_prediction(model, image_path):
    test_image = image.load_img(image_path, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image = test_image * (1./255) # Pamiętaj o skalowaniu takim samym jak w generatorze

    prediction = model.predict(test_image)
    if prediction[0][0] > 0.5:
        print(f'Prediction: face detected with probability {prediction[0][0]}')
    else:
        print(f'Prediction: no face detected with probability {1 - prediction[0][0]}')


### Wywołanie funkcji predykcji

In [ ]:
make_prediction(saved_model, 'ścieżka_do_nowego_testowego_obrazu.jpg')